In [183]:
import numpy as np
from numpy import fft
import pandas as pd
import matplotlib.pyplot as plt

In [184]:
# path to the dataset
path = '/kaggle/input/store-sales-time-series-forecasting/'

In [185]:
# read training data
data_train = pd.read_csv(path + 'train.csv')

In [186]:
families = list(data_train['family'].unique())
stores = list(data_train['store_nbr'].unique())

In [187]:
# The extrapolation function below is cited from 
# https://gist.github.com/tartakynov/83f3cd8f44208a1856ce with a few modification
def fourierExtrapolation(x, n_predict):
    n = x.size
    n_harm = 112                    # number of harmonics in model
    t = np.arange(0, n)
    p = np.polyfit(t, x, 1)         # find linear trend in x
    x_notrend = x - p[0] * t        # detrended x
    x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
    f = fft.fftfreq(n)              # frequencies
    indexes = list(range(n))
    # sort indexes by power specturm, higher -> lower
    indexes.sort(key = lambda i: x_freqdom[i] * np.conj(x_freqdom[i]) / n, reverse = True)
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(x_freqdom[i]) / n   # amplitude
        phase = np.angle(x_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig + p[0] * t

In [188]:
n_predict = 16
for i in stores:
    for j in families:
        data_filtered = data_train[data_train['store_nbr'] == i]
        data_filtered = data_filtered[data_filtered['family'] == j]
        extrapolation = fourierExtrapolation(np.array(data_filtered['sales']), n_predict)
        print(extrapolation, i, j)

[1.6942901  2.11438507 2.75688581 ... 4.90481658 5.06863043 6.42150871] 1 AUTOMOTIVE
[0. 0. 0. ... 0. 0. 0.] 1 BABY CARE
[2.1035163  1.54665223 1.33157196 ... 3.81870955 4.70539111 3.62404743] 1 BEAUTY
[ 427.03760517  845.81402054  960.04286973 ... 2929.74510746 2341.48311699
 2470.86326984] 1 BEVERAGES
[ 0.38660887 -0.08151489 -0.23678687 ...  0.76922497  0.18210341
  0.16556147] 1 BOOKS
[119.99402209 305.21006942 280.59251377 ... 447.94940011 411.91551837
 455.77316156] 1 BREAD/BAKERY
[19.46807581 -9.67485255 13.62221259 ... 13.59277158 28.05199567
 22.29706032] 1 CELEBRATION
[500.29675684 793.29585957 842.81074601 ... 693.37341087 671.57890572
 944.03030145] 1 CLEANING
[221.16029722 412.31745208 434.83039121 ... 788.57081812 726.44984984
 883.9363698 ] 1 DAIRY
[ 70.64162111 117.86567923 132.9027991  ... 171.24654978 129.56992372
 156.15898998] 1 DELI
[ 87.23914586 189.90791606 157.2002951  ... 147.20824926 115.64141321
 176.26419785] 1 EGGS
[ 37.1043111  -65.20726784  -8.53455723 ..